## Gaussian Process Regression

The purpose of this notebook is to introduce **Gaussian Process Regression**. The notebook will introduce in the order of statistical explanation, implementations, and hyperparameter tuning. Many contents of this notebook will based on the prerequisite concepts explained in the *fundamental.ipynb* file.
- **Mathematic Derivation & Definition**
    1. Definition of Gaussian Process (GP)
    2. Definition of Gaussian Process Regression (GPR)
    3. Pros/Cons of GPR
    4. Origination of GPR
    5. Derivation of GPR
    6. Kernel Function in GPR
    7. Sizes of terms
    8. Simple Illustration of GPR
- **Gaussian Process Regression Visualization**
    1. GP From Scratch Code
    2. GP interactive visualization
    3. The role of hyperparameters
    4. GP Using Libraries
- **Gaussian Process Hyperparameter Tuning**



### **Mathematic Derivation & Definition**

#### 1.Definition of Gaussian Process(GP)
A **Gaussian Process (GP)** is a stochastic process that defines a distribution over **functions** rather than just over points. Formally, a GP is a collection of **random variables** where any finite subset follows a **multivariate Gaussian distribution**. A gaussian process is represented as:
$$f\sim \mathcal{GP}(\mu(x),k(x,x'))$$
- $f$ is the function that follows the GP (which is also our target when using GP to make predictions)
- A gaussian process, like gaussian distribution, are defined by only two terms: **mean function** $\mu(x)$ and **kernel (covariance) function** $k(x,x')$.
- The mean function $\mu(x)$ represents the **Expected Value** of the function $f$ given any input $x$.
- The **kernel (covariance) function** $k(x,x')$ captures how function values at different inputs $x$ and $x'$ are related.- GP assumes that function values near each other are correlated in a way dictated by the covariance function (key for generating smooth prediction).

The terminologies seem intimidating, but a gaussian process can be easily understood by the plot below:
<div style="text-align: center"> <img src="https://www.lancaster.ac.uk/stor-i-student-sites/thomas-newman/wp-content/uploads/sites/37/2022/05/Gaussian-process-with-noise.svg" alt="Drawing" width="500"/> </div>

The plot shown here involves training datas ($\times$), some functions, and a shade area. In a gaussian distribution of a random variable (explained in the *fundamental* notebook), we model the distribution of datapoints. Here in GP, we model **the distribution of functions that go through (or nearby, controlled by a hyperparameter explained later) training points**. The shaded area represents the distribution of the **function**, and the functions shown above are some subsets of the distribution. The **bold** function shown in the plot represents the value of **mean function $\mu(x)$**, the most possible function value. The shaded area is dictated by the **kernel(covariance) function**.

#### 2.Definition of Gaussian Process Regression(GPR)
Gaussian Process Regression (GPR) is a Bayesian, nonparametric regression method in which we model an unknown function $f(x)$ as being **drawn from a Gaussian process**. 
Note that gaussian process is **NOT** a machine learning model for prediction. It only describes **Distribution of a function**. Gaussian process can be used to peform multiple tasks, including regression and classification. Here, we focus on **Gaussian Process Regression**, a supervised machine learning model that is cabable of predicting function regardless of its form using gaussian process while also providing uncertainty of the prediction. 

Recall that a regression is a model the relationship between input variables (features $X$) and a **continuous** output variable (Label $Y$):
$$Y=f(X)+\epsilon,\,\epsilon\sim\mathcal{N}(0,\sigma^2)$$

Recall that in the case where the function has a fixed form (linear regression), we use true bayesian prediction to obtain a predictive distribution of label $\hat{Y}$:
$$P(\hat{Y}|X,Y,\hat{X})=\int_{\theta} P(\hat{Y}|\theta,X,Y)P(\theta|X,Y)d\theta$$
The predictive distribution involves a prior $P(\theta)$ over parameter $\theta$, which we assumed to be gaussian in the previous example:
$$P(\theta)\sim \mathcal{N}(0,\tau^2I)$$

**In GPR**, the goal is to predict the function $f$ directly, regardless of its form. Thus, assuming a prior over parameter $\theta$ is **unrealistic**, since the form of $\theta$ is unknown. Instead, we put a **GP prior** over the function which we try to predict. (you can think of that before we have an assumption of $\theta$, now we have an assumption of $f$. Since the distribution of a function is a gaussian process, we use notation $\mathcal{GP}$ instead of $P$.)
$$ \begin{align}
P(\hat{Y}|X,Y,\hat{X})&=\int_{f} P(\hat{Y}|f)P(f|X,Y)df \\
f&\sim \mathcal{GP}(\mu,k)
\end{align}$$


#### 2. Pros/Cons of GPR
##### Advantages of GPR
1. GPR is very powerful in predicting functions in **any form**. Unlike other models that stick with one particular function form (like linear regression for linear model), GP learns the function from the data.
2. GPR uses **Bayesian Approach**, incorporating prior knowledge and allow for principled Bayesian inference, making them adaptable to different domains (for reference MAP/True bayesian prediction in fundamental notebook).
3. GPR is a probabilistic model, whose output not only provides the most probable value (mean function), but also **uncertainty** of the value.
4. GPR does **not require a large dataset**, which is very friendly to the situation where data collection is time-consuming and expensive.
5. The choice of **kernel function** allows GPs to capture various data patterns, such as periodicity, smoothness, or sudden changes.
##### Disadvantages of GPR
1. GPR is computationally **expensive and complex**. This will be explained in later parts.
2. GPR does not work ideally with **large datasets ($n>10^6$)**, primarily because its complexity.
3. GPR does not work ideally with **high dimensional settings**.
4. Choosing kernel functions could be tricky, and it dictates the overall performance of GP.
5. Hyperparameter tuning is challenging.

#### 3. Origination of GPR
The origination of GPR as a powerful model to predict any arbitraty shape function $f$  with uncertainty comes from several aspects.
##### Bayesian Approach
As introduced and explained in the *fundamental* notebook, **bayesian approach** is very powerful at making predictions of model parameters (MLE and MAP). In addition, a **true bayesian prediction** provides uncertainty on top of the prediction itself, making it very useful in many usecase where uncertainty is necessary to obtain. Therefore, gaussian process natually emerges as a model using the bayesian approach.
##### Universality of Gaussian Distribution
Bayesian Approach relies on previous assumptions/knowledge of the distribution (**Prior**) to infer the posterior. This means choosing a **reliable** prior is critical. As shown in the *fundamental* notebook, a **gaussian prior** is the popular choice, due to the universality of gaussian process, and most importantly the **central limit theorem(CLT)**. This property also extends to **gaussian process prior**.

In addition, **operations** among gaussian distributions (addition, multiplication, integration, etc) result in **another gaussian** in most cases. 

These two reasons make choosing a gaussian process prior reasonable. Therefore, a GPR places a gaussian process prior in the model (why it named "gaussian" process).
##### Closed Form of Predictive Distribution
In true bayesian prediction, the **Predictive Distribution** is calculated by integrating over all parameter $\theta$ in the case of fixed function form regression; and integrating over all function $f$ inthe case of gaussian process regression:
$$\begin{align}
P(\hat{Y}|X,Y,\hat{X})&=\int_{\theta} P(\hat{Y}|\theta,X,Y)P(\theta|X,Y)d\theta \\
P(\hat{Y}|X,Y,\hat{X})&=\int_{f} P(\hat{Y}|f)P(f|X,Y)df
\end{align}$$
The indefinite integral is **computationally complex**, and rarely has a **closed form**. In the case of gaussian distribution, a **closed form is available**. This is the key why a GPR can leverage true bayesian prediction as its approach to peform modeling.

#### 4.Derivation of GPR
Recall that for a regression problem, our goal is that given training feature $X$ and training label$Y$, we want to find the unknown function that maps $X$ to $Y$:
$$Y=f(X)+\epsilon, \,\epsilon\sim\mathcal{N}(0,\sigma^2)$$
As Stated above, GPR uses bayesian approach to infer the predictive distribution of function $f(x)$:
$$P(\hat{Y}|X,Y,\hat{X})=\int_{f} P(\hat{Y}|f)P(f|X,Y)df$$

##### Gaussian Conditioning Formula
In the derivation of the predictive distribution, the **Gaussian Conditioning Formula** is frequently used to bypass the cpmlex explicit calculation (specifically the integral).

For two multivariate gaussian random Variable $A$ and $B$:
$$\begin{align}
A&\sim\mathcal{N}(\mu_A,\Sigma_{AA})
B&\sim\mathcal{N}(\mu_B,\Sigma_AA)
\end{align}$$
The joint distribution between $A$ and $B$ is:
$$\begin{bmatrix} A \\ B  \end{bmatrix}\sim\mathcal{N}
\begin{pmatrix}
\begin{bmatrix} \mu_A \\ \mu_B \\ \end{bmatrix} &&
\begin{bmatrix} \Sigma_{AA} && \Sigma_{AB} \\ \Sigma_{BA} && \Sigma_{BB} \end{bmatrix}
\end{pmatrix}
$$
And the conditional distribution $P(B|A)$ is:
$$P(B|A)\sim\mathcal{N}(mu_B+\Sigma_{BA}\Sigma_{AA}^{-1}(A-\mu_A),
        \Sigma_{BB}-\Sigma_{BA}\Sigma_{AA}^{-1}\Sigma_{AB})$$
##### What is the posterior $P(f|X,Y)$
This step is very similar to the posterior function in bayesian linear regression, we use bayes's theorem to infer:
$$P(f|X,Y)\propto P(Y|f)P(f|X)$$
The only difference is that here the prior is a **GP Prior** over $f$:
$$P(f|X)\sim\mathcal{GP}(m(X),K(X,X))$$
where
- $m(X)$ is the mean function evaluated at $X$, and <span style="color: red;">It is assumed that $m(X)=0$ in the gaussia prior.</span>
- $K(X,X) is the **Covariance Matrix**, where entry $K_{ij}=k(X_i,X_j)$, $k(x,x')$ being the chosen kernel function.

$P(Y|f)$ is just the function value with noise:
$$P(Y|f)\sim\mathcal{GP}(m(X),K(X,X)+\sigma^2I)
$$

The **joint distritbuion** of $f(X)$ and $Y$ is:
$$\begin{bmatrix} f(X) \\ Y  \end{bmatrix}\sim\mathcal{N}
\begin{pmatrix}
\begin{bmatrix} m(X) \\ m(X) \\ \end{bmatrix} &&
\begin{bmatrix} K(X,X) && K(X,X) \\ K(X,X) && K(X,X)+\sigma^2I \end{bmatrix}
\end{pmatrix}
$$

Using the **Gaussian Conditioning Formula**,
$$\begin{align}
P(f|X,Y)&=P(f(X)|Y)\sim\mathcal{N}(\mu_f,\Sigma_f) \notag \\
\mu_f &= K(X,X)[K(X,X)+\sigma^2I]^{-1}(Y-m(X)) \notag \\
&=K(X,X)[K(X,X)+\sigma^2I]^{-1}Y \\
\Sigma_f &= K(X,X)-K(X,X)[K(X,X)+\sigma^2I]^{-1}K(X,X)
\end{align}$$

##### What is $P(\hat Y|f)$
The model prediction is just the function output:
$$P(\hat Y|f)\sim\mathcal{N}(m(\hat X),K(\hat X,\hat X))$$

##### Predictive Distribution
To derive the predictive distribution, we don't have to explicitly calculate the integral, because the result of the integral is analytically equal to another gaussian conditioning (both terms under integration are gaussian). The joint distribution of $Y$ and $f(\hat X)$ is:
$$\begin{bmatrix} Y \\ f(\hat X) \end{bmatrix}\sim\mathcal{N}
\begin{pmatrix}
\begin{bmatrix} m(X) \\ m(\hat X) \\ \end{bmatrix} &&
\begin{bmatrix} K(X,X)+\sigma^2I && K(\hat X ,X) \\ K (\hat X,X) && K(\hat X,\hat X) \end{bmatrix}
\end{pmatrix}
$$
Using the **Gaussian Conditioning Formula**,
$$\begin{align}
P(\hat Y|X,Y,\hat X)&=P(f(\hat X)|Y)\sim\mathcal{N}(\mu_*,\Sigma_*) \notag \\
\mu_* &= K_*^T[K+\sigma^2I]^{-1}Y \notag \\
\Sigma_* &= K_{**}-K_*^T[K+\sigma^2I]^{-1}K_*
\end{align}$$
Couple Notes:
- $K=K(X,X)$ is the **Training Kernel Matrix**
- $K_*=K(X,\hat X)$ is the **Training& Testing Covariance Matrix**
- $K_{**}=K(\hat X,\hat X)$ is the **Testing Kernel Matrix**

#### 6.Kernel Function in GPR
A kernel function (also called a covariance function) defines the similarity between data points in Gaussian Process Regression. It specifies how much influence one point has on another, determining the structure of the function space that GPR models. It is extremely important in **GPR** because it controls **smoothness, periodicity, and variability** of the function learned.With that said, choosing an appropriate kernel function is critical.

Before we dive in to determine the form of kernel function, we need to identify the requirements on the kernel function. We recognize that for kernel matrix $K$, where $K_ij=k(X,X')$:
1. The Diagonal of the matrix must represent the variance of the distribution. This is because we need to satisfy gaussian prior:
$$K_{ii}=k(X_i,X_i)=Var(X_i)$$
2. $K$ is always **Positive semi-definite**, because variance/covariance **cannot be negative**.
3. If $X_i$ and $X_j$ are very **independent**, i.e.$X_i$ and $X_j$ are very different from each other, $K(X_i,X_j)=0$.
4. If $X_i$ and $X_j$ are very **similar**, $K(X_i,X_j)>0$.

##### Radial Basis Function (RBF) Kernel
A very popular kernel function that satisfies all the requirements mentioned above is the **RBF** kernel, which defines similarity with eucleadian distance:
$$K(X_i,X_j)=\tau \exp \frac{-||X_i-X_j||^2}{2l^2}$$
where
- $tau$ is the prefactor (coefficient)of the model. Sometimes it is also called $\sigma^2$, but may cause confusion because there is another $\sigma^2$ in the model which is the observation noise variance.
- $l$ is the lengthscale. Controls how far the influence of each data point extends. A larger $l$ means smoother functions, while a smaller $l$ allows rapid changes.
- $||X_i-X_j||^2$ is the eucleadian distance between the two feature vectors.